In [20]:
import pandas as pd
import pyarrow.parquet as pq
import airportsdata 

### Data Collection

In [ ]:
airports = airportsdata.load()
edinburgh = airports['EGPH']
glasgow = airports['EGPF']
prestwick = airports['EGPK']
type(edinburgh)

In [13]:
dataset = pq.ParquetDataset('c:\\Users\\lukas\\OneDrive - Imperial College London\\0_Sustainable Transport\\2021_COP26',
                            ) # filters=[('columnName', 'in', filterList)]
df = dataset.read(use_threads=True).to_pandas()

In [9]:
df.columns

Index(['callsign', 'icao_address', 'flight_number', 'tail_number',
       'aircraft_type_icao', 'aircraft_engine_type', 'origin_airport',
       'origin_airport_name', 'origin_country', 'destination_airport',
       'destination_airport_name', 'destination_country',
       'scheduled_departure_time_utc', 'scheduled_arrival_time_utc',
       'first_waypoint_time', 'last_waypoint_time', 'duration_hours',
       'total_distance_km', 'great_circle_distance_km',
       'longitude_first_waypoint', 'longitude_last_waypoint', 'longitude_min',
       'longitude_max', 'latitude_first_waypoint', 'latitude_last_waypoint',
       'latitude_min', 'latitude_max', 'altitude_first_waypoint',
       'altitude_last_waypoint', 'altitude_min', 'altitude_max', 'n_wypts',
       'n_terrestrial', 'n_satellites', 'n_interpolate', 'n_extrapolate',
       'n_great_circle', 'extrap_trajectory_start', 'extrap_trajectory_end',
       'trajectory_complete', 'great_circle_trajectory', 'est_origin_airport',
       'es

Filter by cop26 duration, airports(origin,destination), unique id(count airplane once) 

In [38]:
cop_airports = ['EGPF','EGPK','EGPH']
# test= df[df['destination_airport'] == 'EGPF']
# test['scheduled_departure_time_utc']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### COP26 - Glasgow
Airport in Glasgow: Glasgow airport (EGPF), Prestwick airport(EGPK)

Airport in Edinburgh: Edinburgh airport (EGPH)

In [19]:
df[df["destination_airport"] == "EGPH"]

,callsign,icao_address,flight_number,tail_number,aircraft_type_icao,aircraft_engine_type,origin_airport,origin_airport_name,origin_country,destination_airport,...,so2_kg,sulphates_kg,nvpm_data_source,nvpm_mass_kg,nvpm_number,nvpm_ei_n_mean,sdr_mean,night_flight,night_segments,pct_night_dist
1636,NPT10K,406EC8,,G-JMCV,B734,Jet,EGSS,London Stansted Airport,GB,EGPH,...,2.418472,0.049357,FA Model,0.219498,2.045562e+18,1.014969e+15,0.000000,True,True,100.0
2296,NPT04N,40761A,,G-NPTX,B734,Jet,EGNX,East Midlands Airport,GB,EGPH,...,2.153291,0.043945,FA Model,0.218706,2.371498e+18,1.321604e+15,0.000000,True,True,100.0
5954,PYN04,40747F,,G-FLYK,BE20,Turboprop,EGSS,London Stansted Airport,GB,EGPH,...,0.439979,0.008979,Constant,0.032265,3.666492e+17,1.000000e+15,0.000000,True,True,100.0
11768,TAY4125,44D1CE,3V4125,OO-TNN,B734,Jet,EBBR,Brussels Airport,BE,EGPH,...,4.170344,0.085109,FA Model,0.307341,3.513269e+18,1.010929e+15,0.000000,True,True,100.0
14938,DHK2134,400663,D02134,G-BMRA,B752,Jet,EGNX,East Midlands Airport,GB,EGPH,...,2.537902,0.051794,FA Model,0.425342,3.789197e+18,1.791652e+15,0.000000,True,True,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6284445,EZY471U,407835,U2489,G-UZLI,A20N,Jet,EGAA,Belfast International Airport,GB,EGPH,...,1.230273,0.025108,ICAO EDB,0.001549,5.577076e+16,5.439844e+13,0.000069,True,True,100.0
6284623,RYR526G,4CAA61,FR818,EI-EXF,B738,Jet,EIDW,Dublin Airport,IE,EGPH,...,1.794680,0.036626,ICAO EDB,0.068181,1.388938e+18,9.287036e+14,0.000069,True,True,100.0
6286886,EZY318Q,405D0F,U2504,G-EZDV,A319,Jet,EGBB,Birmingham International Airport,GB,EGPH,...,1.950779,0.039812,ICAO EDB,0.062294,1.880417e+18,1.156717e+15,0.000069,True,True,100.0
6286953,EZY102T,406752,U2239,G-EZWA,A320,Jet,EGSS,London Stansted Airport,GB,EGPH,...,2.457277,0.050149,ICAO EDB,0.104555,2.178266e+18,1.063746e+15,0.000069,True,True,100.0


### Data visualisation

##### Heatmap

##### Plots (Arrival of flights over time, Load capacity over time, CO2 emissions)